In [ ]:
#libraraies to import 


import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm 
from collections import defaultdict
import matplotlib.pyplot as plt
from PIL import Image 
import imutils
import math
from keras.utils.np_utils import to_categorical
import h5py
import tensorflow
#from keras.models import load_model
from keras.utils import plot_model
from keras.models import Model,load_model,Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Conv2D, MaxPool2D,  Dropout,Flatten, Input, concatenate, Dense
from keras.datasets import mnist,cifar100
from keras.preprocessing.image import Iterator
import keras.backend as K
import scipy
from scipy import ndimage

In [ ]:
#fucntions for data aug and collection

def create_data(X):
    data_dict = {}
    shp =X.shape
    print(shp)
    rot_img = []
    y = []
    for i in X:
        img  = i
        rotation_angle = np.random.randint(36)
        rotated_image =  generate_rotated_image(img, angle=-(rotation_angle*10))
        rotated_image = cv2.resize(rotated_image, (96,96))
        #print('rot img',rotated_image.shape)
        #img_1 = img/255
        #rotated_image_1 = rotated_image/255
        #data_dict['img'].append(img_1)
        #data_dict['rot_img'].append(rotated_image_1)
        rot_img.append(np.array(rotated_image))
        #y.append([np.sin(rotation_angle),np.cos(rotation_angle)])
        y.append(rotation_angle)
        
    
    
    print('rot donee')
    rot_img = np.array(rot_img)
    X1 = X/255
    rot_img1 = rot_img/255
    data_dict['img'] = X1 
    data_dict['rot_img'] = rot_img1
    batch_y = to_categorical(y, 36)
    y1 =np.array(y) 
    y = np.array((180-np.array(y))/180)
    return data_dict,batch_y
def create_data2(X,x1,y):
    data_dict = {}
    x = X
    rot_img = x1
    y = y//10
    
#     shp =X.shape
#     print(shp)
#     rot_img = []
#     y = []
#     for i in X:
#         img  = i
#         rotation_angle = np.random.randint(360)
#         rotated_image =  generate_rotated_image(img, angle=-rotation_angle,size=img.shape)
#         #print('rot img',rotated_image.shape)
#         #img_1 = img/255
#         #rotated_image_1 = rotated_image/255
#         #data_dict['img'].append(img_1)
#         #data_dict['rot_img'].append(rotated_image_1)
#         rot_img.append(rotated_image)
#         y.append(rotation_angle)
    
    
    print('rot donee')
    rot_img = np.array(rot_img)
    X1 = x/255
    #print
    rot_img1 = rot_img/255
    data_dict['img'] = X1 
    data_dict['rot_img'] = rot_img1
    batch_y = to_categorical(y, 36) 
    #y =[np.sin(rotation_angle),np.cos(rotation_angle)] 
    return data_dict,batch_y

def angle_difference(x, y):
    """
    Calculate minimum difference between two angles.
    """
    return 180 - abs(abs(x - y) - 180)
    #return abs(x - y)


def angle_error(y_true, y_pred):
    """
    Calculate the mean diference between the true angles
    and the predicted angles. Each angle is represented
    as a binary vector.
    """
    diff = angle_difference(K.argmax(y_true), K.argmax(y_pred))
    return K.mean(K.cast(K.abs(diff), K.floatx()))

def binarize_images(x):
    """
    Convert images to range 0-1 and binarize them by making
    0 the values below 0.1 and 1 the values above 0.1.
    """
    x /= 255
    x[x >= 0.1] = 1
    x[x < 0.1] = 0
    return x

def angle_error_regression(y_true, y_pred):
    """
    Calculate the mean diference between the true angles
    and the predicted angles. Each angle is represented
    as a float number between 0 and 1.
    """
    return K.mean(angle_difference(y_true * 360, y_pred * 360))

def rotate(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    Source: http://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
    """
    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result


def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.
    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow
    Converted to Python by Aaron Snoswell
    Source: http://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    Source: http://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]


def crop_largest_rectangle(image, angle, height, width):
    """
    Crop around the center the largest possible rectangle
    found with largest_rotated_rect.
    """
    return crop_around_center(
        image,
        *largest_rotated_rect(
            width,
            height,
            math.radians(angle)
        )
    )


def generate_rotated_image(image, angle, size=None, crop_center=True,
                           crop_largest_rect=True):
    """
    Generate a valid rotated image for the RotNetDataGenerator. If the
    image is rectangular, the crop_center option should be used to make
    it square. To crop out the black borders after rotation, use the
    crop_largest_rect option. To resize the final image, use the size
    option.
    """
    height, width = image.shape[:2]
    if crop_center:
        if width < height:
            height = width
        else:
            width = height

    image = rotate(image, angle)

    if crop_largest_rect:
        image = crop_largest_rectangle(image, angle, height, width)

    if size:
        image = cv2.resize(image, size)

    return image

In [ ]:
#loading and combing data


ata = np.load('coco_train2.npy')
print(ata.shape)
data = np.load('coco_train_rot2.npy')
y = np.load('coco_train_y2.npy')
print(data.shape)
print(y.shape)
#print(ata[2].shape)
train = ata[:-65000]
train_rot = data[:-65000]
train_y = y[:-65000]

test = ata[-10000:]
test_rot = data[-10000:]
test_y = y[-10000:]



print('1',train.shape)
print('2',train_rot.shape)
print('3',train_y.shape)
print('4',test.shape)
print('5',test_rot.shape)
print('6',test_y.shape)




dx = ndimage.sobel(ata[764], 0)  # horizontal derivative
dy = ndimage.sobel(ata[764], 1)  # vertical derivative
mag = np.hypot(dx, dy)  # magnitude
mag *= 255.0 / np.max(mag)  # normalize (Q&D)
plt.imshow(mag)
plt.show()

print(y[764])
plt.imshow(ata[764])
# print(X_train[5].shape)
# print(x.shape)
# plt.imshow(X_train[5])
plt.show()
plt.imshow(data[764])
plt.show()

In [ ]:
#model in use 


model_name = 'rotnet_11_1'

(X_train, _), (X_test, _) = mnist.load_data()

kernel_size = (3, 3)
nb_train_samples, img_rows, img_cols = train.shape

input_shape = train.shape[1:]
nb_test_samples = test.shape[0]

print('Input shape:', input_shape)
print(nb_train_samples, 'train samples')
print(nb_test_samples, 'test samples')


inp1 = Input((96,96,1), name='img')
# xx = Conv2D(128, kernel_size, activation='relu')(inp1)
# xx = MaxPool2D(pool_size=(2, 2))(xx)



inp2 = Input((96,96,1),name='rot_img')
# xx2 = Conv2D(128, kernel_size, activation='relu')(inp2)
# xx2 = MaxPool2D(pool_size=(2, 2))(xx2)

deepVO = concatenate([inp1, inp2],axis=-1)
deepVO = Conv2D(64, kernel_size, activation='relu')(deepVO)
deepVO = Conv2D(64, kernel_size, activation='relu')(deepVO)
deepVO = MaxPool2D(pool_size=(2, 2))(deepVO)
deepVO = Dropout(0.25)(deepVO)
deepVO = Flatten()(deepVO)
deepVO = Dense(128, activation='relu')(deepVO)
deepVO = Dropout(0.25)(deepVO)
output =  Dense(36,activation='softmax')(deepVO)



model = Model(inputs=[inp1, inp2], outputs=output)

model.summary()

In [ ]:
#compile and fitting the model






opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(loss= tensorflow.keras.losses.sparse_categorical_crossentropy,
#               optimizer='adam')

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[angle_error,tensorflow.keras.metrics.CategoricalAccuracy()])#tensorflow.keras.metrics.CategoricalAccuracy()

# training parameters
batch_size = 2
nb_epoch = 50

output_folder = 'models'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# callbacks
checkpointer = ModelCheckpoint(
    filepath=os.path.join(output_folder, model_name + '.hdf5'),
    save_best_only=True
)
#early_stopping = EarlyStopping(patience=2)
tensorboard = TensorBoard()

x,y=create_data2(train,train_rot,train_y)

# training loop
model.fit(
    x=[x['img'],x['rot_img']],
    y=y,
    steps_per_epoch=nb_train_samples / batch_size,
    epochs=nb_epoch,
    validation_data=create_data2(test,test_rot,test_y),
    validation_steps=nb_test_samples / batch_size,
    verbose=1,
    callbacks=[checkpointer, tensorboard]
)
 
